# nParticles = 1

This is a control for the train of particles. It's just a squirmer. One happy squirmer.

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import copy
import auxiliary as aux

mpl.rc("figure", dpi=150)
mpl.rc("figure", figsize=(4,4))

plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'xtick.labelsize':15,
    'ytick.labelsize':15,
    'axes.labelsize':20,
})

import os
import re

%reload_ext autoreload
%autoreload 2

In [ ]:
# extra stuff
from matplotlib.ticker import FormatStrFormatter, ScalarFormatter
import math
from scipy.spatial import cKDTree  # For finding nearest neighbors

# formatting
formatter = ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-2, 2))  # Sets limits for when to use scientific notation

In [ ]:
# output directory
src_n = 0
dataDir = f'../data.lbm/0{src_n}/'
# dataDir = f'../src/{src_n}/output.lbm/'

# all ticks are found using regular expressions
files = os.listdir(dataDir)
ticks = []

pattern = r'fluidTrj_(\d+)\.csv'

for file in files:
    match = re.search(pattern, file)
    if match:
        ticks.append(int(match.group(1)))
        
ticks.sort()

len(ticks)

# Fluid

In [ ]:
id = -1
fluidDf = pd.read_csv(dataDir + f"fluidTrj_{ticks[id]}.csv").set_index(["id_x","id_y"]).sort_index()
particleDf = pd.read_csv(dataDir+"particlesTrj.csv")

In [ ]:
title = f'$t = {fluidDf.time.values[-1]:0.1f}$'+' $\\mu \\mathrm{s}$'
# fig, axes = aux.fluidOverview(fluidDf, particleDf, title)
fig, axes = aux.fluidOverview(
    fluidDf.query("(coordinate_x-10)**2 < 20**2 and coordinate_y**2 < 20**2"),
    particleDf, 
    title,
)
plt.show()

# particles

The interactions between the particles are:
- a linear bond for each train link
- WCA potentials

So the distance between them must remain fairly constant.

In [ ]:
nParticles = len(particleDf.particleId.unique())
particleTime = particleDf.time.unique()

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(6,2.7))

for Id in np.arange(nParticles):
    axes[0].plot(particleTime, particleDf.query(f"particleId == {Id+1}").position_x.values, )
axes[0].set_title("position -- $x$")
    
for Id in np.arange(nParticles):
    axes[1].plot(particleTime, np.sqrt(
            particleDf.query(f"particleId == {Id+1}").velocity_x.values**2 + 
            particleDf.query(f"particleId == {Id+1}").velocity_y.values**2
        )
    )
axes[1].set_title("speed -- $|\\mathbf{u}|$")
axes[1].set_ylim(0,1e-3)
axes[1].yaxis.tick_right()  # Moves y-axis ticks to the right
axes[1].set_yticks([0, 0.5e-3, 1e-3])
axes[1].set_yticklabels(["0", "", "$10^{-3}$"])
    
plt.show()

# Stability analysis

In [ ]:
def avgSpeed(df):
    return np.sqrt(df.fluidVelocity_x**2 + df.fluidVelocity_y**2).mean()

In [ ]:
try:
    avgFluidSpeed 
except NameError:
    avgFluidSpeed = []
    
try:
    time 
except NameError:
    time = []

for tick in ticks[len(time)::]:
    auxDf = pd.read_csv(dataDir + f"fluidTrj_{tick}.csv")
    avgFluidSpeed.append(avgSpeed(auxDf))
    time.append(auxDf.time.values[0])
    print(f'\r{tick}', end='', flush=True)
print('\r'+' '*10, end='', flush=True)

In [ ]:
fig, ax = plt.subplots(figsize=(3,2.7))

ax.plot(time, avgFluidSpeed)
ax.set_xlabel("$t ~ (\\mu \\mathrm{s})$")
ax.set_ylabel("$|u| ~ (\\mu \\mathrm{m} / \\mu \\mathrm{s})$")

plt.show()

# Animation

In [ ]:
# necessary directories are created
outputDir = f"frames.{src_n}"
os.system(f'mkdir -p anims')
os.system(f'mkdir {outputDir}')

# the data is read and frames are saved
particleDf = pd.read_csv(dataDir+"particlesTrj.csv")
for tickId in np.arange(len(ticks)):
    fluidDf = pd.read_csv(dataDir + f"fluidTrj_{ticks[tickId]}.csv").set_index(["id_x","id_y"]).sort_index()
    title = f'$t = {fluidDf.time.values[-1]:.0f}$'+' $\\mu \\mathrm{s}$'
    fig, axes = aux.fluidOverview(fluidDf.query("coordinate_x**2 < 20**2 and coordinate_y**2 < 20**2"), particleDf, title)

    plt.savefig(f"{outputDir}/{tickId}.png", format="png", dpi=300, bbox_inches="tight")
    plt.close()
    print(f'\r{tickId}', end='', flush=True)
print('\r'+' '*10, end='', flush=True)

# ffmpeg is used to create the animation
mkAnimSh = f'ffmpeg -framerate 20 -i {outputDir}/%d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -loglevel quiet anims/src_{src_n}.mp4'
os.system(mkAnimSh)

# the directory with the frames is erased
os.system(f'rm -r {outputDir}')